In [182]:
import torch
import torchvision
import torchvision.transforms as transforms
import idx2numpy
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import PIL
import os
import random
import optuna
import joblib

In [1]:
!git init

Initialized empty Git repository in /home/adsayan/Programming/python/poly/.git/


In [3]:
!git add emotion*.*

fatal: pathspec 'emotion*.py' did not match any files


In [168]:
trainimages = []
trainlabels = []
index = 0
mode = "Train"
Map= {}
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    Map[index]=emotion

    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            filepath =f'emotionfolder/{mode}/{emotion}/{file}'
            trainimages.append(filepath)
            trainlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [141]:
testimages = []
testlabels = []
index = 0
mode = 'Test'
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            img = f'emotionfolder/{mode}/{emotion}/{file}'
            testimages.append(img)
            testlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [142]:
#split is rougly 60/40 right now, need to make it ~80/10/10
trainimages.extend(testimages)
trainlabels.extend(testlabels)

trainimages_final = []
trainlabel_final = []
testimages_final = []
testlabel_final = []
valimages_final = []
vallabels_final = []

In [145]:

for i in range(0, len(trainimages)-1):
    num = random.randint(1,10)

    if 1<=num<=8:
        trainimages_final.append(trainimages[i])
        trainlabel_final.append(trainlabels[i])

    elif num==9:
        testimages_final.append(trainimages[i])
        testlabel_final.append(trainlabels[i])

    else:
        valimages_final.append(trainimages[i])
        vallabels_final.append(trainlabels[i])
        
    

In [148]:
target_size = (96,96)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [149]:
train_transform = transforms.Compose([
    #transforms.ToPILImage(),

    transforms.RandomResizedCrop(target_size),
    
    transforms.RandomHorizontalFlip(),
    transforms.RandomAdjustSharpness(2),
    transforms.RandomAutocontrast(),
    transforms.ColorJitter(brightness=0.2,saturation=0.2),

    transforms.ToTensor(),
    transforms.Normalize(mean, std)

])

test_transform = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),

    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
    
    

In [150]:
class EmotionDataset(Dataset):
    def __init__(self, images,labels, transform):
        self.images= images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        PilImage = Image.open(self.images[idx])
        img = self.transform(PilImage)
        label = self.labels[idx]
        return img,label

In [157]:
trainDataset = EmotionDataset(trainimages_final, trainlabel_final, train_transform)
valDataset = EmotionDataset(valimages_final, vallabels_final, test_transform)
testDataset = EmotionDataset(testimages_final, testlabel_final, test_transform)
trainloader = DataLoader(trainDataset, shuffle=True, batch_size=128)
testloader = DataLoader(testDataset, shuffle=True, batch_size=128)
valloader = DataLoader(valDataset, shuffle=True, batch_size=128)

In [158]:
class CnnArchitecture(nn.Module):#96,96
    def __init__(self, output_size, is_grayscale=True):
        super().__init__()
        self.output_size= output_size
        self.input_channels = 1 if is_grayscale else 3
        self.c1 = nn.Conv2d(in_channels=self.input_channels, out_channels=64, kernel_size=3, padding=1)
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, padding=0)

        self.c2 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1)

        self.output = nn.Linear(128*24*24, self.output_size)

    def forward(self,x):
        x = self.c1(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.c2(x)
        x = self.activation(x)
        x=self.pool(x)
        x = torch.flatten(x, 1,-1)

        return self.output(x)
    

In [159]:
model = CnnArchitecture(8, False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [162]:
for epoch in range(1):
    model.train()
    train_loss= 0

    for x, y in trainloader:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        train_loss+=loss.item()
        loss.backward()
        optimizer.step()

    model.eval()
    error_loss = 0
    total = 0
    correct = 0
    with torch.no_grad():
        for x,y in valloader:
            y_pred = model(x)
            loss = criterion(y_pred,y)
            error_loss+=loss.item()
            _pred, index = torch.max(y_pred.data, 1)

            total+=y.size(0)#where y.size(0) is batch size
            correct+= (index==y).sum().item()

    avg_train_loss = train_loss/len(trainloader)
    avg_val_los = error_loss / len(valloader)
    accuracy = 100 * correct / total

    print(f"Epoch num is {epoch}")
    print(f"train loss is {avg_train_loss}")
    print(f"test loss is {avg_val_los}")
    print(f'accuracy is {accuracy}')
    print('------------------------')
    
    
            

Epoch num is 0
train loss is 1.973477362965544
test loss is 1.7323530068000157
accuracy is 35.411227154047
------------------------


In [165]:
joblib.dump(model, "trial_emotion_reco.pkl")

['trial_emotion_reco.pkl']

In [166]:
model = joblib.load( "trial_emotion_reco.pkl")


def predictEmotion(mode, Mapping, image):
    if not isinstance(image, PIL.JpegImagePlugin.JpegImageFile):
        raise ValueError("expected JPEG format Image for prediction image")

    Imagetensor = test_transform(img)
    Image = Imagetensor.unsqueeze(0)

    with torch.no_grad():
        y_pred = model(Image)

    confidence = 
        

In [ ]:
#to do optimization of model
def objective(trial):
    num_conv = trial.suggest_categorical("num_conv", [1,2,3,4,5])
    hidden_size = trial.suggest_categorical("hidden_size", [16,32,64,128])
    batch_size = trial.suggest_categorical("batch_size", [1,4,16,32,64,128])
    lr = trial.suggest_categorical('lr', [1e-2,1e-3,1e-4,1e-5])
    optimizer = trial.suggest_categorical("optimizer", ['Adam', 'SGD'])
    #define rest later on
    
    trainDataset = EmotionDataset(trainimages_final, trainlabel_final, train_transform)
    valDataset = EmotionDataset(valimages_final, vallabels_final, test_transform)
    testDataset = EmotionDataset(testimages_final, testlabel_final, test_transform)
    trainloader = DataLoader(trainDataset, shuffle=True, batch_size=batch_size)
    testloader = DataLoader(testDataset, shuffle=True, batch_size=batch_size)
    valloader = DataLoader(valDataset, shuffle=True, batch_size=batch_size)

    model = CnnArchitecture(8, False)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    

    
    

In [176]:
path = trainimages[15]

In [177]:
img = Image.open(path)

In [186]:
imgq = test_transform(img)

In [189]:
imgq.unsqueeze(0).shape

torch.Size([1, 3, 96, 96])